## Setup

In [138]:
EXPERIMENT_NAME = 'lightweight-component-with-reusable-component-training-3'

In [139]:
#REVERSE_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-dataflow-tfdv:6554e133dd453c62aea05ebb57a04f897c11d070'
REVERSE_IMAGE = 'gcr.io/spielwiese-sascha-heyer/kubeflow/training/component:latest'

## Define the pipeline

In [140]:
import kfp
from kfp import compiler
import kfp.components as comp
import kfp.dsl as dsl
from kfp import gcp

In [141]:
reverse_operation = kfp.components.load_component_from_url('https://storage.googleapis.com/spielwiese-sascha-heyer/test6/component.yaml')


In [142]:
def concat(first: str, second: str) -> str:
   return first + second

In [143]:
concat_operation = comp.func_to_container_op(concat)

In [144]:
@dsl.pipeline(
  name='test',
  description='test'
)
def text_transformation_pipeline(first, second):
    
    concat_task = concat_operation(first, second) 
    
    #reverse_operation = dsl.ContainerOp(
    #            name="reverse",
    #            image=REVERSE_IMAGE,
    #            command=["python3", "/pipelines/component/src/component.py"],
    #            arguments=[
    #                '--input-text', concat_task.output,
    #            ],
    #            file_outputs={
    #                'text': '/text.txt',
    #            }
    #        )
    
    reverse = reverse_operation(text=concat_task.output).apply(kfp.gcp.use_gcp_secret('user-gcp-sa')) 
    
    

In [145]:
pipeline_func = text_transformation_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'

import kfp.compiler as compiler
compiler.Compiler().compile(text_transformation_pipeline, pipeline_filename)

In [146]:
#Specify pipeline argument values
arguments = {'first': 'dess', 'second': 'erts'}

#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()
#experiment = client.create_experiment(EXPERIMENT_NAME)
experiment = client.get_experiment(experiment_name=EXPERIMENT_NAME)
print(experiment)

{'created_at': datetime.datetime(2019, 6, 14, 7, 54, 30, tzinfo=tzlocal()),
 'description': None,
 'id': '6952f36d-5628-4976-9c48-7f5db5238664',
 'name': 'lightweight-component-with-reusable-component-training-3'}


In [147]:
#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)